In [1]:
%%capture
!pip install ipywidgets

In [2]:
%%capture
# ! pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
! pip install git+https://github.com/keepsimpler/sunyata
! pip install pytorch-lightning
! pip install lightning-bolts
! pip install einops

In [3]:
import os, math, random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder, VisionDataset
from torchvision.datasets.folder import default_loader


import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from sunyata.pytorch.arch.base import BaseModule, Residual
from sunyata.pytorch.arch.foldnet import FoldNet, FoldNetCfg, FoldNetRepeat, FoldNetRepeat2, Block

In [4]:
cfg = FoldNetCfg(
    block = Block,
    hidden_dim = 256,
    num_layers = 32,
    kernel_size = 5,
    patch_size = 6,
    num_classes = 59,
    fold_num = 1,
    drop_rate = 0.2,
    
    batch_size = 128,
    num_epochs = 100,
    learning_rate = 5e-3,
    optimizer_method = "AdamW",
    weight_decay = 0.1,
    learning_rate_scheduler= "LinearWarmupCosineAnnealingLR",
    warmup_epochs = 10,  
    warmup_start_lr = 1e-5,
)

In [5]:
train_root_dir = '/notebooks/PDR2018/train/'
valid_root_dir = '/notebooks/PDR2018/valid/'

In [6]:
import pandas as pd
valid_cls_to_img = pd.read_json(valid_root_dir + 'AgriculturalDisease_validation_annotations.json')
train_cls_to_img = pd.read_json(train_root_dir + 'AgriculturalDisease_train_annotations.json')


In [7]:
updated = valid_cls_to_img['disease_class'] > 45
valid_cls_to_img.loc[updated, 'disease_class'] = valid_cls_to_img.loc[updated, 'disease_class'] - 2

In [8]:
updated = train_cls_to_img['disease_class'] > 45
train_cls_to_img.loc[updated, 'disease_class'] = train_cls_to_img.loc[updated, 'disease_class'] - 2

In [9]:
train_samples = train_cls_to_img[['image_id', 'disease_class']].values.tolist()
valid_samples = valid_cls_to_img[['image_id', 'disease_class']].values.tolist()
len(train_samples), len(valid_samples)

(31716, 4539)

In [10]:
train_samples = [ (train_root_dir + "resized_256/" + img_fname, idx) for img_fname, idx in train_samples]
valid_samples = [ (valid_root_dir + "resized_256/" + img_fname, idx) for img_fname, idx in valid_samples]

In [11]:
class VisionDatasetWrap(VisionDataset):
    def __init__(self, root_dir, samples, transform=None):
        super().__init__(root_dir, transform=transform)
        self.samples =samples
        self.loader = default_loader
        
    def __getitem__(self, idx: int):
        img_path, target = self.samples[idx]
        img = self.loader(img_path)
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, target
    
    def __len__(self):
        return len(self.samples)

In [21]:
192 + 60

252

In [22]:
train_transform = torchvision.transforms.Compose([
    # torchvision.transforms.RandomCrop(256),
    torchvision.transforms.Resize((252,252)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandAugment(num_ops=2, magnitude=12),
    torchvision.transforms.ColorJitter(0.3, 0.3, 0.3),
    torchvision.transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = torchvision.transforms.Compose(
    [
        # torchvision.transforms.CenterCrop(256),
        torchvision.transforms.Resize((252,252)),
        torchvision.transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)


In [23]:
train_dataset = VisionDatasetWrap(train_root_dir, transform=train_transform, samples=train_samples)
val_dataset = VisionDatasetWrap(valid_root_dir, transform=val_transform, samples=valid_samples)

In [24]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=8)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=8)


In [25]:
model = FoldNetRepeat(cfg)

pl.utilities.model_summary.summarize(model,max_depth=1)

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 2.4 M 
1 | embed  | Sequential | 28.4 K
2 | digup  | Sequential | 15.2 K
--------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.579     Total estimated model params size (MB)

In [26]:
trainer = pl.Trainer(
    # progress_bar_refresh_rate=10,
    precision=16,
    max_epochs=cfg.num_epochs,
    accelerator='gpu',
    devices=1,
    enable_checkpointing=True,
    # detect_anomaly=True,
    logger=CSVLogger("lightning_logs/", name="convmixer"),
    callbacks=[
        LearningRateMonitor(logging_interval="epoch"),
        ModelCheckpoint(save_top_k=2, monitor="val_accuracy", mode='max')
    ],
)    


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/usr/local/lib/python3.9/dist-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/usr/local/lib/python3.9/dis

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [19]:
# trainer.fit(model, 
#             train_dataloader, val_dataloader,
#             ckpt_path = "/notebooks/lightning_logs/convmixer/version_377/checkpoints/epoch=80-step=27540.ckpt",
#            )


In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 2.4 M 
1 | embed  | Sequential | 28.4 K
2 | digup  | Sequential | 15.2 K
--------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
4.789     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [24]:
cfg = FoldNetCfg(
    block = Block,
    hidden_dim = 256,
    num_layers = 32,
    kernel_size = 5,
    patch_size = 6,
    num_classes = 12,
    fold_num = 2,
    drop_rate = 0.,
    
    batch_size = 128,
    num_epochs = 100,
    learning_rate = 5e-3,
    optimizer_method = "AdamW",
    weight_decay = 0.1,
    learning_rate_scheduler= "LinearWarmupCosineAnnealingLR",
    warmup_epochs = 10,  # 2//5 * num_epoches
    warmup_start_lr = 1e-5,
)

In [25]:
model = FoldNetRepeat(cfg)

pl.utilities.model_summary.summarize(model,max_depth=1)

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 2.4 M 
1 | embed  | Sequential | 28.4 K
2 | digup  | Sequential | 6.2 K 
--------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.543     Total estimated model params size (MB)

In [26]:
trainer = pl.Trainer(
    # progress_bar_refresh_rate=10,
    precision=16,
    max_epochs=cfg.num_epochs,
    accelerator='gpu',
    devices=1,
    enable_checkpointing=True,
    # detect_anomaly=True,
    logger=CSVLogger("lightning_logs/", name="convmixer"),
    callbacks=[LearningRateMonitor(logging_interval="epoch")],
)    


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | ModuleList | 2.4 M 
1 | embed  | Sequential | 28.4 K
2 | digup  | Sequential | 6.2 K 
--------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
4.771     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
